In [ ]:
pip install estaty==0.1.0

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 75.9 MB/s eta 0:00:00
  Created wheel for carto: filename=carto-1.11.3-py3-none-any.whl size=35058 sha256=c0d68725feeae45f27d6427e3200cc7a290a835a9904

In [ ]:
import geopandas as gpd
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from estaty.analysis.action import Analyzer
from estaty.data_source.action import DataSource
from estaty.main import EstateModel
from estaty.preprocessing.action import Preprocessor
import warnings

# **1. Вариант парсера**
работает через раз + долго + естЬ ограничения


In [ ]:
geolocator = Photon(user_agent="measurements")
ageolocator = Nominatim(user_agent="my-test-app")

In [ ]:
warnings.filterwarnings('ignore')


def launch_proximity_analysis_for_school_objects(lat, lon):
    point_for_analysis = {'lat': lat, 'lon': lon}
    osm_source = DataSource('osm', params={'category': 'school'})

    osm_reprojected = Preprocessor('reproject', params={'to': 'auto'}, from_actions=[osm_source])

    analysis = Analyzer('distance', params={'network_type': 'walk', 'visualize': True, 'color': '#C67E00',
                                            'edgecolor': 'black', 'title': 'Schools'},
                        from_actions=[osm_reprojected])

    model = EstateModel().for_property(point_for_analysis, radius=2000)
    founded_routes = model.compose(analysis)

    print(founded_routes.lines)
    print(f'Min length: {founded_routes.lines["Length"].min():.2f}, m')
    print(f'Mean length: {founded_routes.lines["Length"].mean():.2f}, m')
    print(f'Max length: {founded_routes.lines["Length"].max():.2f}, m')


In [ ]:
with open('адреса.txt') as f:
    file = open('адреса.txt', "r+", encoding='utf-8')
    addresses = [address[:-1] for address in file.readlines() if address != '']
    for address in addresses:
      ageolocator.geocode(address)
      lat = location.latitude
      lon = location.longitude
      launch_proximity_analysis_for_school_objects(lat, lon)

AttributeError: module 'fiona' has no attribute 'path'

In [ ]:
address = "1600 Amphitheatre Parkway, Mountain View, CA"
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut
from shapely.geometry import Point
import geopandas as gpd
import osmnx as ox

# **2. версия парсера**
те же проблемы

In [ ]:


# Создаем геолокатор с уникальным user_agent
geolocator = Nominatim(user_agent="my_geocode_app")

# Добавляем ограничитель частоты запросов
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)


# Функция для геокодирования адресов
def geocode_address(address):
    try:
        location = geocode(address)  # Используем RateLimiter
        if location:
            return Point(location.longitude, location.latitude)
        else:
            return None
    except GeocoderTimedOut as e:
        print(f"Ошибка тайм-аута при геокодировании адреса {address}: {e}")
        return None
    except Exception as e:
        print(f"Ошибка при геокодировании адреса {address}: {e}")
        return None


# Функция для вычисления минимального расстояния до объектов
def calculate_min_distance(point, category_tag):
    if point is None:
        return None
    try:
        # Радиус поиска (в метрах)
        radius = 5000
        bbox = ox.utils_geo.bbox_from_point((point.y, point.x), dist=radius)
        category_objects = ox.features.features_from_bbox(
            bbox=bbox, tags={category_tag: True}
        )

        if category_objects.empty:
            print(f"Нет объектов для категории {category_tag} в радиусе {radius} м.")
            return None

        distances = category_objects.geometry.apply(lambda obj: obj.distance(point))
        return distances.min()
    except Exception as e:
        print(f"Ошибка при вычислении расстояния до категории {category_tag}: {e}")
        return None


# Загрузка данных из Excel
input_file = "offers.xlsx"
output_file = "updated_offers.xlsx"

df = pd.read_excel(input_file)
print("Файл успешно загружен.")

# Геокодирование адресов
print("Преобразование адресов в координаты...")
df["geometry"] = df["Адрес"].apply(geocode_address)

# Преобразование в GeoDataFrame
gdf_addresses = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Категории объектов для анализа
categories = {
    "Детские сады": {"amenity":"childcare"},
    "Школы": {"amenity":"school"},
    "Бары": {"amenity":"bar"},
    "Парки": {"leisure":"park"},
    "Водоемы": "natural:water",
    "Больницы": "amenity:hospital"
}

# Вычисление минимальных расстояний для каждой категории
print("Вычисление минимальных расстояний до объектов...")
for category_name, category_tag in categories.items():
    print(f"Обработка категории: {category_name}")
    gdf_addresses[category_name] = [
        calculate_min_distance(geometry, category_tag) if geometry else None
        for geometry in gdf_addresses.geometry
    ]

# Сохранение обновленного файла
gdf_addresses.to_excel(output_file, index=False)
print(f"Результат сохранен в {output_file}.")

# **3. версия парсера**

другое все, но расстояние 0 метров


In [ ]:
import osmnx as ox

def find_nearest_place(start_address, place_type):
    # Получаем координаты начального адреса
    start_point = ox.geocode(start_address)

    # Находим ближайшие места заданного типа
    tags = {'amenity': place_type}
    places = ox.geometries_from_point(start_point, dist=1000, tags=tags)

    if places.empty:
        return "Не удалось найти места указанного типа в радиусе 1000 метров."

    # Находим ближайшее место
    nearest_place = places.iloc[0]
    nearest_place_location = (nearest_place.geometry.centroid.y, nearest_place.geometry.centroid.x)

    # Находим маршрут от начального адреса до ближайшего места
    orig_node = ox.distance.nearest_nodes(G, start_point[1], start_point[0])
    dest_node = ox.distance.nearest_nodes(G, nearest_place_location[1], nearest_place_location[0])

    route = ox.shortest_path(G, orig_node, dest_node, weight='length')

    # Получаем длину маршрута
    route_length = ox.distance.euclidean_dist_vec(start_point[0], start_point[1], nearest_place_location[0], nearest_place_location[1])

    return f"Ближайшее {place_type}: {nearest_place['name']}. Расстояние: {route_length:.2f} метров."

# Пример использования
start_address = 'Тверская улица, 12С6'
place_type = 'school'
result = find_nearest_place(start_address, place_type)
print(result)


<ipython-input-131-536247133d1e>:9: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  places = ox.geometries_from_point(start_point, dist=1000, tags=tags)
/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to Ge

Ближайшее school: Школа №1113. Расстояние: 0.01 метров.


<ipython-input-131-536247133d1e>:25: FutureWarning: The `euclidean_dist_vec` function has been renamed `euclidean`. Calling `euclidean_dist_vec` will raise an error starting in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  route_length = ox.distance.euclidean_dist_vec(start_point[0], start_point[1], nearest_place_location[0], nearest_place_location[1])


# **4. версия парсера**

 все хорошо, но 83 км до школы...


In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point

def find_nearest_place(start_address, place_type):
    # Получаем координаты начального адреса
    start_point = ox.geocode(start_address)

    # Создаём GeoDataFrame для начальной точки
    start_point_gdf = gpd.GeoDataFrame(
        {'geometry': [Point(start_point[1], start_point[0])]},
        crs='EPSG:4326'
    ).to_crs('EPSG:3857')  # Переводим в метры для расчёта расстояний


    # Находим ближайшие места заданного типа
    tags = {'amenity': place_type}
    places = ox.geometries_from_point(start_point, dist=1000, tags=tags)

    if places.empty:
        return "Не удалось найти места указанного типа в радиусе 1000 метров."

    # Переводим места в ту же CRS и вычисляем расстояния
    places = places.to_crs('EPSG:3857')
    places['distance'] = places.geometry.centroid.distance(start_point_gdf.geometry.iloc[0])

    # Находим ближайшее место
    nearest_place = places.loc[places['distance'].idxmin()]
    nearest_place_location = (nearest_place.geometry.centroid.y, nearest_place.geometry.centroid.x)

    # Находим маршрут от начального адреса до ближайшего места
    orig_node = ox.distance.nearest_nodes(G, start_point[1], start_point[0])
    dest_node = ox.distance.nearest_nodes(G, nearest_place_location[1], nearest_place_location[0])

    route = ox.shortest_path(G, orig_node, dest_node, weight='length')

    # Получаем длину маршрута
    route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))

    # Возвращаем информацию о ближайшем месте
    name = nearest_place.get('name', 'Без названия')
    return f"Ближайшее {place_type}: {name}. Расстояние по маршруту: {route_length:.2f} метров."

# Пример использования
start_address = 'Тверская улица, 12С6'
place_type = 'school'
result = find_nearest_place(start_address, place_type)
print(result)


/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyproj/transformer.py:817: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(
<ipython-input-129-e73aa7d02c88>:18: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  places = ox.geometries_from_point(start_point, dist=1000, tags=tag

Ближайшее school: Гимназия №1520 им. Капцовых. Расстояние по маршруту: 83828.06 метров.


<ipython-input-129-e73aa7d02c88>:38: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))


# **5. Последняя работающая версия.**


upd: [документация](https://osmnx.readthedocs.io/en/latest/getting-started.html)

In [ ]:
pip install estaty==0.1.0

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 30.3 MB/s eta 0:00:00
  Created wheel for carto: filename=carto-1.11.3-py3-none-any.whl size=35058 sha256=a5690f48facd152e9d047c19880efafa1339b1498f32b

In [ ]:
pip install osmnx

In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import csv

In [ ]:
#G = ox.graph_from_place('Moscow, Russia', network_type='walk')
#ox.io.save_graphml(G)

#делаем и сохраняем граф дорог Москвы, чтобы не создавать по новой каждый раз (12 минут как никак)

In [ ]:
G = ox.io.load_graphml("drive/MyDrive/Colab Notebooks/graph.graphml")

In [ ]:
def find_nearest_place(start_address, place_type):
    start_point = ox.geocode(start_address)
    tags = place_type
    places = ox.geometries_from_point(start_point, dist=5000, tags=tags)

    if places.empty:
        return "Не удалось найти места указанного типа в радиусе 5000 метров."

    start_point_gdf = gpd.GeoDataFrame(
        {'geometry': [Point(start_point[1], start_point[0])]},
        crs='EPSG:4326'
    )
    start_point_gdf = start_point_gdf.to_crs(places.crs)

    places['distance'] = places.geometry.centroid.distance(start_point_gdf.geometry.iloc[0])
    nearest_place = places.loc[places['distance'].idxmin()]
    nearest_place_location = nearest_place.geometry.centroid

    orig_node = ox.distance.nearest_nodes(G, start_point[1], start_point[0])
    dest_node = ox.distance.nearest_nodes(G, nearest_place_location.x, nearest_place_location.y)

    route = ox.shortest_path(G, orig_node, dest_node, weight='length')

    route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))

    walking_speed_mps = 1.39
    walking_time_seconds = route_length / walking_speed_mps
    walking_time_minutes = round(walking_time_seconds / 60)

    name = nearest_place.get('name', 'Без названия')
    return walking_time_minutes

In [ ]:
# Пример использования
start_address = 'Тверская улица, 12С6'
place_type = {"amenity":"bar"}
result = find_nearest_place(start_address, place_type)
print(result)

<ipython-input-6-d8424ba6d674>:4: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  places = ox.geometries_from_point(start_point, dist=5000, tags=tags)
/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
<ipython-input-6-d8424ba6d674>:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  places['distance'] = places.geometry.centroid.distance(start_point_gdf.geometry.iloc[0])
<ipython-input-6-d8424

1


<ipython-input-6-d8424ba6d674>:24: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))


далее я чисто скопировала часть из кода Руслана, крч это ваша часть (только подкорректировала доступные категории(все проверила, все работают) + удалила пару строчек, которые и так у меня в функции есть)




In [ ]:
dff = pd.read_csv('drive/MyDrive/Colab Notebooks/offers-_1_.csv')
header = [['Площадь, м2', 'Этаж', 'Количество комнат', 'Ремонт', 'Тип', 'Метро', 'Парки', 'Школы', 'Детские сады', 'Больницы', 'Бары', 'Парковки','Цена','Ссылка']]
#df = pd.DataFrame(header)
#df.to_csv(r'drive/MyDrive/Colab Notebooks/offers_new.csv', mode='a', header=False, index=False)
all_new = []
n = 176 #номер строки, с которой начинать, в случае ошибки (неправильный адрес, долгая обработка, нету в радиусе данных категорий)
for i in range(n, len(dff)):
  address = dff.loc[i]['Адрес']
  rooms = dff.loc[i]['Количество комнат']
  tip = dff.loc[i]['Тип']
  metro = dff.loc[i]['Метро']
  plosh = dff.loc[i]['Площадь, м2']
  stage = dff.loc[i]['Дом']
  parkov = dff.loc[i]['Парковка']
  price = dff.loc[i]['Цена']
  remont = dff.loc[i]['Ремонт']
  link = dff.loc[i]['Ссылка на объявление']
  all_new.append([plosh, stage, rooms, remont, tip, metro])

  categories = {
    "Парки": {"leisure":"park"},
    "Школы": {"amenity":"school"},
    "Детские сады": {"amenity":"childcare"},
    "Больницы": {"amenity":"hospital"},
    "Бары": {"amenity":"bar"}
  }
  for category_name in categories.items():
    result = find_nearest_place(address, category_name[1])
    all_new[i-n].append(result)

  all_new[i-n].append(parkov)
  all_new[i-n].append(price)
  all_new[i-n].append(link)
  df = pd.DataFrame([all_new[i-n]], columns=header)
  df.to_csv(r'drive/MyDrive/Colab Notebooks/offers_new.csv', mode='a', header=False, index=False)

<ipython-input-6-d8424ba6d674>:4: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  places = ox.geometries_from_point(start_point, dist=5000, tags=tags)
/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoD

если захотите еще добавить категорий, то вот список возможных (не факт что все будут работать):


```
# Основные категории для тега amenity:

Образование:

school — школа.
college — колледж.
university — университет.
kindergarten — детский сад.

Еда и напитки:
bar — бар.
restaurant — ресторан.
cafe — кафе.
fast_food — фастфуд.

Медицина:
hospital — больница.
clinic — клиника.
pharmacy — аптека.
dentist — стоматология.

Развлечения и досуг:
cinema — кинотеатр.
theatre — театр.
library — библиотека.

Покупки и услуги:
marketplace — рынок.
bank — банк.
atm — банкомат.

Транспорт:
parking — парковка.
bus_station — автовокзал.
fuel — заправочная станция.

Другие популярные категории:

Досуг (тег leisure):
park — парк.
playground — детская площадка.
sports_centre — спортивный центр.
stadium — стадион.

Шопинг (тег shop):
supermarket — супермаркет.
mall — торговый центр.
convenience — продуктовый магазин.

Туризм (тег tourism):
hotel — отель.
museum — музей.
attraction — достопримечательность.

Землепользование (тег landuse):
cemetery — кладбище.
forest — лес.
```

